In [1]:
from utils import (
    authenticate_and_initialize, load_panama_geometry, get_embeddings_collection, get_year_embedding,
    export_thumbnail, cosine_similarity, compute_mean_embedding, create_geemap, VIS_SIMILARITY_VIRIDIS
)
import ee, json
authenticate_and_initialize()
pa_geometry = load_panama_geometry()
embeddings = get_embeddings_collection(pa_geometry)
# Use 2024 embedding image as reference year
emb24 = get_year_embedding(2024, embeddings)
band_names = emb24.bandNames()
print('Loaded 2024 embedding with', band_names.size().getInfo(), 'bands.')

Loaded 2024 embedding with 64 bands.


In [2]:
# Helper functions specific to this notebook
def get_point_embedding(image: ee.Image, point: ee.Geometry, scale: int = 20) -> ee.Array:
    feature = image.sample(region=point, scale=scale, numPixels=1).first()
    if feature is None: raise ValueError('No pixel sampled at point')
    d = feature.toDictionary()
    return ee.Array(d.values(image.bandNames()))

def similarity_map_for_point(image: ee.Image, point: ee.Geometry, scale: int = 20) -> ee.Image:
    vec = get_point_embedding(image, point, scale)
    return cosine_similarity(image, vec)

def similarity_map_for_mean(image: ee.Image, fc_samples: ee.FeatureCollection) -> ee.Image:
    mean_vec = compute_mean_embedding(fc_samples, image.bandNames())
    return cosine_similarity(image, mean_vec)

SIM_VIS = VIS_SIMILARITY_VIRIDIS

## 1. Individual Points of Interest
Copper Mine: (lat 8.85197, lon -80.64478) -> EE point [-80.64478, 8.85197]
Dump: (lat 9.053050, lon -79.565120) -> EE point [-79.565120, 9.053050]

In [3]:
mine_point = ee.Geometry.Point([-80.64478, 8.85197])
dump_point = ee.Geometry.Point([-79.565120, 9.053050])
mine_sim = similarity_map_for_point(emb24, mine_point)
dump_sim = similarity_map_for_point(emb24, dump_point)
print('Prepared similarity maps for mine and dump.')

Prepared similarity maps for mine and dump.


In [4]:
mine_url = export_thumbnail(mine_sim.clip(pa_geometry), pa_geometry, 'similarity_mine_2024', SIM_VIS, dimensions=2048)
dump_url = export_thumbnail(dump_sim.clip(pa_geometry), pa_geometry, 'similarity_dump_2024', SIM_VIS, dimensions=2048)
mine_url, dump_url

similarity_mine_2024: https://earthengine.googleapis.com/v1/projects/gen-lang-client-0315720004/thumbnails/ea9ff5f9ee688e9b0a9f88d621edd5d1-f089323f8f90ea80fdc59dae81206d2c:getPixels
similarity_dump_2024: https://earthengine.googleapis.com/v1/projects/gen-lang-client-0315720004/thumbnails/cf4f4fcf79cbb932abf6deb9ed02e93d-fcd225b208d5e0da3a475a7dd2fe0f6e:getPixels


('https://earthengine.googleapis.com/v1/projects/gen-lang-client-0315720004/thumbnails/ea9ff5f9ee688e9b0a9f88d621edd5d1-f089323f8f90ea80fdc59dae81206d2c:getPixels',
 'https://earthengine.googleapis.com/v1/projects/gen-lang-client-0315720004/thumbnails/cf4f4fcf79cbb932abf6deb9ed02e93d-fcd225b208d5e0da3a475a7dd2fe0f6e:getPixels')

## 2. Water Sample Points (Feature Collection)
Operations: 
- Load `waters.geojson` as a FeatureCollection.
- Compute similarity for 'gatun' point.
- Compute mean embedding over all canal = 'true' points and similarity.
- Compute mean embedding over all canal = 'false' points and similarity.

In [25]:
with open('./assets/geo/waters.geojson') as f:
    waters_geojson = json.load(f)
    
features = []
for feat in waters_geojson.get('features', []):
    geom = ee.Geometry(feat.get('geometry'))
    props = feat.get('properties', {})
    features.append(ee.Feature(geom, props))
waters_fc = ee.FeatureCollection(features)
print('Total water sample points:', waters_fc.size().getInfo())

# Sample embeddings at each point (scale coarse enough to be stable)
sample_scale = 20
waters_samples = emb24.sampleRegions(collection=waters_fc, scale=sample_scale, geometries=True)
print('Sampled embeddings for water points.')
# Split by canal property (stored as string 'true'/'false')
canal_true_fc = waters_fc.filter(ee.Filter.eq('canal', 'true'))
canal_false_fc = waters_fc.filter(ee.Filter.eq('canal', 'false'))
canal_true_samples = emb24.sampleRegions(collection=canal_true_fc, scale=sample_scale, geometries=True)
canal_false_samples = emb24.sampleRegions(collection=canal_false_fc, scale=sample_scale, geometries=True)

Total water sample points: 8
Sampled embeddings for water points.


In [26]:
# Gatun point similarity
gatun_fc = waters_fc.filter(ee.Filter.eq('name', 'gatun'))
gatun_point = ee.Feature(gatun_fc.first()).geometry()
gatun_sim = similarity_map_for_point(emb24, gatun_point, scale=sample_scale)
# Mean embeddings for canal true/false
mean_true_vec = compute_mean_embedding(canal_true_samples, band_names)
mean_false_vec = compute_mean_embedding(canal_false_samples, band_names)
canal_true_sim = cosine_similarity(emb24, mean_true_vec)
canal_false_sim = cosine_similarity(emb24, mean_false_vec)
print('Computed similarity maps for gatun, canal mean (true), canal mean (false).')

Computed similarity maps for gatun, canal mean (true), canal mean (false).


In [27]:
gatun_url = export_thumbnail(gatun_sim.clip(pa_geometry), pa_geometry, 'similarity_gatun_2024', SIM_VIS, dimensions=2048)
canal_true_url = export_thumbnail(canal_true_sim.clip(pa_geometry), pa_geometry, 'similarity_canal_true_mean_2024', SIM_VIS, dimensions=2048)
canal_false_url = export_thumbnail(canal_false_sim.clip(pa_geometry), pa_geometry, 'similarity_canal_false_mean_2024', SIM_VIS, dimensions=2048)
gatun_url, canal_true_url, canal_false_url

similarity_gatun_2024: https://earthengine.googleapis.com/v1/projects/gen-lang-client-0315720004/thumbnails/0297ff476c23234292ba43226ceb40e1-b05568901f0f050395b1134286011b79:getPixels
similarity_canal_true_mean_2024: https://earthengine.googleapis.com/v1/projects/gen-lang-client-0315720004/thumbnails/7f88c902f8494879a7a41d364d26840b-debcafdd75083c2df3c7be4570803d55:getPixels
similarity_canal_false_mean_2024: https://earthengine.googleapis.com/v1/projects/gen-lang-client-0315720004/thumbnails/c92640e41500e3e5abf4d6a52a36a0c4-b0017855428c32bc95c49ab465e26c56:getPixels


('https://earthengine.googleapis.com/v1/projects/gen-lang-client-0315720004/thumbnails/0297ff476c23234292ba43226ceb40e1-b05568901f0f050395b1134286011b79:getPixels',
 'https://earthengine.googleapis.com/v1/projects/gen-lang-client-0315720004/thumbnails/7f88c902f8494879a7a41d364d26840b-debcafdd75083c2df3c7be4570803d55:getPixels',
 'https://earthengine.googleapis.com/v1/projects/gen-lang-client-0315720004/thumbnails/c92640e41500e3e5abf4d6a52a36a0c4-b0017855428c32bc95c49ab465e26c56:getPixels')

## 3. Interactive Map
Layers added: mine, dump, gatun, canal mean (true), canal mean (false).

In [28]:
Map = create_geemap(center_geometry=pa_geometry, zoom=8)
Map.addLayer(mine_sim.clip(pa_geometry), SIM_VIS, 'Mine Similarity')
Map.addLayer(dump_sim.clip(pa_geometry), SIM_VIS, 'Dump Similarity')
Map.addLayer(gatun_sim.clip(pa_geometry), SIM_VIS, 'Gatun Similarity')
Map.addLayer(canal_true_sim.clip(pa_geometry), SIM_VIS, 'Canal True Mean Similarity')
Map.addLayer(canal_false_sim.clip(pa_geometry), SIM_VIS, 'Canal False Mean Similarity')
Map

Map(center=[8.517242119967118, -80.12094816672555], controls=(WidgetControl(options=['position', 'transparent_…

## Notes on Methodology
- Each pixel embedding is a 64-D vector. Cosine similarity emphasizes directional alignment (pattern similarity) rather than magnitude.
- Point embeddings are sampled at scale 20m for stability. Adjust scale to refine locality vs. representativeness.
- Mean vectors act as centroids of the specified sets of points. They summarize typical embedding characteristics of canal vs. non-canal water samples.
- Interpretation: High similarity areas share embedding spectral/latent characteristics with the reference (e.g., potential environmental or land-cover correlates).